Why DIRECT? in this paper, the threats have been simplified to circular boundaries. In real life, the boundaries will be irregular, and will have no explicit differentiable expression avalible.

In [42]:
from scipydirect import minimize
import numpy as np

# Deriving the cost function.

## Finding an initial cost function

* Consider problem in two dimensions.
* Route consists of start and endpoints with a number of intermediate waypoints.
* The coordinates of the waypoints are the optimization variables.
* We want the distance flown to be minimal, so calculate Euclidian distance of the route, and label it $L$.
* Determine if any of our path lies within a threat, and the length of the path within the threat.

If the route lies within the $i$-th threat for a distance $L_i$ then the cost of the route can be expressed as

$C = L + \displaystyle\sum_{i} \rho _i L_{i}^{p}$

$\rho _i$ is a penalty parameter for the $i$-th threat.

In other words, the cost is the total length of the path, plus the length of each segment within a threat multiplied by the penalty parameter associated with that particular threat.

## Defining "leg" length
We have $n$ waypoints $w_j = (w_{j1}, w_{j2})$ for $j = 1, ..., n$, let the start and end points be $w_0, w_{n+1}$ respectively. We can define the vector $u_j = w_j - w_{j-1}$ which helps us defined the length of any given leg as 

$l_j = \sqrt{(u_{j1}^2 + u_{j2}^2)}$

## Determining threat violations

We  will use linear interpoliation to estimate where leg $j$ intersects with threat $i$. 

Consider $\sigma _{max}$ to be the maximum step size for use in sampling any leg of a route. This means that the number of samplings points along leg $j$ is $K_{max} = \lceil l_j / \sigma _{max} \rceil$. 



In [43]:
# Cost function
def cost(x, threats):
    pass

# Circular Threats from paper (page 6)
#                     Centre,    radius
threats = [(np.array([75,  29]),    9   ),
           (np.array([80,  66]),    15  ),
           (np.array([95,  115]),   22.5),
           (np.array([135, 131]),   20  ),
           (np.array([140, 67.5]),  37.5),
           (np.array([197, 125]),   30  ),]